In [1]:
from server.lib.BackTest import BackTest
from server.lib.models import EMAGoldenCrossoverModel, TestStopLossModel, BenchMarkModel
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
Test = BackTest(TestStopLossModel)
Bench = BackTest(BenchMarkModel)

Test.run()
Bench.run()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [4]:
bt = BackTest(EMAGoldenCrossoverModel)
bt.run()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
